In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from env import api_key
import requests
import json
from acquire import *
from bs4 import BeautifulSoup
import os
import re
from prepare import *

In [2]:
df = pd.read_csv("master_df.csv")

In [3]:
df = prepare_bills_for_processing(df)

In [4]:
more_stopwords = ['secretary','united','states','sente','house','representative',
                   'representatives','fiscal','year','shall','adding','end','paragraph',
                   'made','available','prebody','subsection','day','date','submit','described',
                   'may','congress','following','new','enactment','code','section','assembled',
                   'b','amended','short','title' ]

In [5]:
df['lem'] = df.bill_text.apply(clean_text)

In [8]:
df['model'] = df.lem.apply(join)

In [9]:
df.head()

,sponsor,party,bill_text,lem,model
0,"Torres Small, Xochitl",D,\n\n\n \n To require the Secretary of Homelan...,"[require, homeland, security, develop, plan, i...",require homeland security develop plan increas...
1,"Sewell, Terri A.",D,\n\n\n \nTo amend title XI of the Social Secur...,"[amend, xi, social, security, act, clarify, pa...",amend xi social security act clarify parameter...
2,"Finkenauer, Abby",D,\n\n\n \n To repeal the funding authorization ...,"[repeal, funding, authorization, sunset, total...",repeal funding authorization sunset total fund...
3,"Casten, Sean",D,\n\n\n \nTo direct the Federal Energy Regulato...,"[direct, federal, energy, regulatory, commissi...",direct federal energy regulatory commission fi...
4,"LaMalfa, Doug",R,\n\n\n \n To establish a categorical exclusio...,"[establish, categorical, exclusion, electric, ...",establish categorical exclusion electric utili...


In [10]:
train, X_train, y_train, X_val, y_val, X_test, y_test = split_data(df, 'party')

In [11]:
X_train.head()

,sponsor,bill_text,lem,model
25840,"Frelinghuysen, Rodney P.",\n\n\n \n To suspend temporarily the duty...,"[suspend, temporarily, duty, 3amino12propanedi...",suspend temporarily duty 3amino12propanediol a...
24423,"Foxx, Virginia",\n\n\n \n Of inquiry requesting the President...,"[inquiry, requesting, president, directing, ed...",inquiry requesting president directing educati...
16270,"Scott, Rick",\n\n\n \n To allow National Flood Insurance Pr...,"[allow, national, flood, insurance, program, p...",allow national flood insurance program policyh...
21136,"Hickenlooper, John W.",\n\n\n \n To provide better access to retireme...,"[provide, better, access, retirement, plan, sm...",provide better access retirement plan small bu...
11697,"Dingell, Debbie",\n\n\n \n To amend the Energy Policy and C...,"[amend, energy, policy, conservation, act, mod...",amend energy policy conservation act modify de...


In [12]:
all_bill_words = clean_text(' '.join(df['bill_text']))

In [13]:
democrat_words = clean_text(' '.join(df[df['party'] == 'D']['bill_text']))

In [14]:
republican_words = clean_text(' '.join(df[df['party'] == 'R']['bill_text']))

In [15]:
all_bill_bigrams = pd.Series(nltk.ngrams(all_bill_words, 2))
all_bill_bigrams

0                (require, homeland)
1               (homeland, security)
2                (security, develop)
3                    (develop, plan)
4                   (plan, increase)
                      ...           
26072686      (withdrawn, warehouse)
26072687    (warehouse, consumption)
26072688         (consumption, 15th)
26072689                 (15th, act)
26072690              (act, ltallgt)
Length: 26072691, dtype: object

In [16]:
top_bill_bigrams = all_bill_bigrams.value_counts().head(40)

In [17]:
top_bill_bigrams

(42, usc)                     39539
(act, 42)                     26089
(1, act)                      23289
(public, health)              22856
(america, 1)                  22250
(security, act)               21618
(health, care)                20538
(act, cited)                  19672
(public, law)                 18478
(state, local)                17859
(higher, education)           17493
(social, security)            17308
(homeland, security)          16395
(meaning, given)              16092
(given, term)                 15868
(law, enforcement)            15463
(internal, revenue)           15064
(indian, tribe)               15000
(act, 2)                      14295
(small, business)             14057
(purpose, america)            13864
(health, service)             13736
(attorney, general)           13678
(federal, agency)             13590
(revenue, 1986)               13149
(day, act)                    13132
(authorized, appropriated)    12420
(health, human)             

In [18]:
democrat_bigrams = pd.Series(nltk.ngrams(democrat_words, 2))


In [19]:
top_democrat_bigrams =democrat_bigrams.value_counts().head(40)

In [20]:
top_democrat_bigrams 

(42, usc)                     30830
(act, 42)                     20579
(public, health)              18006
(health, care)                15889
(security, act)               15642
(1, act)                      15027
(higher, education)           14352
(state, local)                13757
(public, law)                 13666
(america, 1)                  13320
(indian, tribe)               12675
(social, security)            12578
(meaning, given)              12423
(given, term)                 12230
(act, cited)                  12147
(law, enforcement)            10841
(small, business)             10732
(health, service)             10724
(internal, revenue)           10528
(authorized, appropriated)    10226
(attorney, general)            9977
(day, act)                     9424
(revenue, 1986)                9340
(health, human)                9208
(federal, agency)              9202
(human, service)               9198
(institution, higher)          9108
(homeland, security)        

In [21]:
republican_bigrams = pd.Series(nltk.ngrams(republican_words, 2))

In [22]:
top_republican_bigrams =republican_bigrams.value_counts().head(40)

In [23]:
top_republican_bigrams

(america, 1)            8800
(42, usc)               8372
(1, act)                8147
(act, cited)            7428
(homeland, security)    7261
(security, act)         5750
(act, 42)               5256
(act, 2)                5235
(purpose, america)      5169
(public, health)        4748
(public, law)           4734
(law, enforcement)      4597
(health, care)          4431
(social, security)      4428
(federal, agency)       4355
(internal, revenue)     4288
(state, local)          4052
(people, republic)      3727
(attorney, general)     3677
(day, act)              3662
(republic, china)       3610
(revenue, 1986)         3593
(meaning, given)        3587
(given, term)           3556
(national, security)    3331
(local, government)     3317
(small, business)       3297
(higher, education)     3033
(ltallgt, amend)        2981
(15, usc)               2943
(health, service)       2931
(1, striking)           2897
(service, act)          2775
(joint, resolution)     2754
(clause, ii)  

In [24]:
dem_counts = pd.Series(democrat_words).value_counts()

In [25]:
rep_counts = pd.Series(republican_words).value_counts()

In [26]:
all_counts = pd.Series(all_bill_words).value_counts()

In [27]:
word_freq = pd.concat([dem_counts, rep_counts, all_counts], axis = 1)
word_freq.head(40)

,0,1,2
act,250491.0,90594.0,343081
1,219086.0,80462.0,301295
2,181426.0,67036.0,249936
program,141582.0,33156.0,175478
service,124381.0,35043.0,160319
state,120042.0,37763.0,158386
c,110433.0,36337.0,147606
including,109187.0,27716.0,137491
health,105155.0,26996.0,133023
3,103356.0,36016.0,140166


In [28]:
republican_quatgrams = pd.Series(nltk.ngrams(republican_words, 4))

In [29]:
top_republican_quatgrams =republican_quatgrams.value_counts().head(40)

In [30]:
top_republican_quatgrams

(america, 1, act, cited)                                          6878
(purpose, america, 1, act)                                        4372
(social, security, act, 42)                                       2183
(security, act, 42, usc)                                          2182
(public, health, service, act)                                    2138
(immigration, nationality, act, 8)                                1664
(nationality, act, 8, usc)                                        1663
(service, act, 42, usc)                                           1432
(health, service, act, 42)                                        1417
(99, inserting, numerical, sequence)                              1364
(ii, 99, inserting, numerical)                                    1364
(later, 180, day, act)                                            1265
(government, people, republic, china)                             1170
(free, change, change, 12)                                        1156
(feder

In [31]:
democrat_quatgrams = pd.Series(nltk.ngrams(democrat_words, 4))
top_democrat_quatgrams =democrat_quatgrams.value_counts().head(40)
top_democrat_quatgrams

(america, 1, act, cited)                                          10824
(social, security, act, 42)                                        7758
(security, act, 42, usc)                                           7753
(purpose, america, 1, act)                                         7296
(public, health, service, act)                                     6237
(act, 1965, 20, usc)                                               5572
(education, act, 1965, 20)                                         5557
(higher, education, act, 1965)                                     4722
(service, act, 42, usc)                                            4221
(health, service, act, 42)                                         4147
(authorization, appropriationsthere, authorized, appropriated)     4054
(center, disease, control, prevention)                             3579
(later, 180, day, act)                                             3522
(federal, food, drug, cosmetic)                                 